### Run notebook 02_LoL_predictor_Data_Preparation before running this one, as it prepares the data!

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import yaml

In [2]:
with open('config.yml', mode="r") as f:
    config = yaml.safe_load(f)

import wandb
wandb.login()

wandb.init(
        project="lol-predictor",
        entity="mschon",
        config=config['train'],
        group='pytorch',
        mode='online'
    )

wandb: Currently logged in as: mschon. Use `wandb login --relogin` to force relogin


In [25]:
pd.set_option('display.max_columns', 500)
DATA_FOLDER = '../data/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
train = pd.read_csv(f"{DATA_FOLDER}/train.csv")
test = pd.read_csv(f"{DATA_FOLDER}/test.csv")
validation = pd.read_csv(f"{DATA_FOLDER}/validation.csv")

In [27]:
train.shape

(6375, 22)

In [28]:
test.shape

(1482, 22)

In [29]:
validation.shape

(1482, 22)

In [9]:
trainX = train.drop(columns={"blueWins"})
trainy = train.blueWins

In [10]:
testX = test.drop(columns={"blueWins"})
testy = test.blueWins

In [11]:
nHidden = config['train']['inputNeurons']

model = nn.Sequential(
    nn.Linear(16, nHidden[0]),
    nn.ReLU(),
    nn.Linear(nHidden[0], nHidden[1]),
    nn.ReLU(),
    nn.Linear(nHidden[1], nHidden[2]),
    nn.ReLU(),
    nn.Linear(nHidden[2], nHidden[3]),
    nn.ReLU(),
    nn.Linear(nHidden[3], 1),
    nn.Sigmoid()
    # Softmax
)

In [12]:
trainX = torch.tensor(trainX.values)
testX = torch.tensor(testX.values)

In [13]:
trainy = torch.tensor(trainy.values)
testy = torch.tensor(testy.values)

# https://stackoverflow.com/a/60440460/12342419
trainy = trainy.type(torch.LongTensor)
testy = testy.type(torch.LongTensor)

In [14]:
def compute_loss(y_hat, y):
    return nn.BCELoss()(y_hat, y)

In [15]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
trainX = trainX.to(torch.float32).to(device)
testX = testX.to(torch.float32).to(device)
trainy = trainy.to(torch.float32).to(device)
testy = testy.to(torch.float32).to(device)

In [18]:
trainy = trainy.reshape((trainy.shape[0], 1))

In [19]:
model

Sequential(
  (0): Linear(in_features=16, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=32, bias=True)
  (7): ReLU()
  (8): Linear(in_features=32, out_features=1, bias=True)
  (9): Sigmoid()
)

In [20]:
model.to(device)
print(device)

cpu


In [21]:
torch.cuda.is_available()

False

In [22]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

RuntimeError: No CUDA GPUs are available

In [23]:
num_epochs = config['train']['epochs']

for n in range(num_epochs):
    model.train()
    y_pred = model(trainX)
    loss = compute_loss(y_pred, trainy)
    
    if n % (num_epochs / 100) == 0:
        print(n)
        #print(y_pred)
        print(accuracy_fn(y_true=trainy, y_pred=torch.round(y_pred)))
        accuracy = accuracy_fn(y_true=trainy, y_pred=torch.round(y_pred))
        wandb.log({'epoch': n, 'accuracy': accuracy, 'loss': loss})

    optimizer.zero_grad()
    loss.backward()
    #loss.backward(retain_graph=False)
    optimizer.step()
print(loss)

0
49.77811782708493
50
72.37949502677888
100
72.94567712318286
150
73.11400153022188
200
72.93037490436113
250
73.58837031369548
300
73.8638102524866
350
74.44529456771232
400
74.4299923488906
450
74.91966335118592
500
75.24100994644223
550
75.34812547819433
600
75.73068094873757
650
75.40933435348126
700
76.17444529456772
750
76.40397857689365
800
76.89364957918899
850
76.90895179801072
900
76.90895179801072
950
76.98546289211936
1000
77.50573833205814
1050
77.5822494261668
1100
77.62815608263199
1150
77.76587605202755
1200
78.11782708492731
1250
78.19433817903597
1300
78.25554705432288
1350
78.46977811782708
1400
78.48508033664882
1450
78.51568477429227
1500
78.97475133894415
1550
78.86763580719204
1600
79.1124713083397
1650
79.23488905891354
1700
79.18898240244836
1750
79.54093343534812
1800
79.35730680948737
1850
79.29609793420046
1900
79.92348890589135
1950
79.54093343534812
2000
79.8928844682479
2050
80.07651109410865
2100
80.26013771996939
2150
80.19892884468248
2200
80.0
2250
8

In [ ]:
y_pred.shape

In [ ]:
torch.round(y_pred[:10])

In [ ]:
testy[:10]

In [ ]:
def predict(model, test_loader):
    all_preds = []
    all_preds_raw = []
    all_labels = []

    for batch in test_loader:
        batch.x = torch.tensor(batch.x)
        batch.x = batch.x.reshape((-1, *batch.x.shape[2:]))
        batch.to(device)  
        pred = model(torch.tensor(batch.x).float(), 
                        #batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch) 

        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    return all_preds, all_preds_raw, all_labels

In [ ]:
torch.round(testX[0])

In [ ]:
testy

In [ ]:
len(model(testX))

In [ ]:
model(testX)[0:10]

In [ ]:
testy[:10]

In [ ]:
lol = model(testX[20:30])
print(lol)
#print(lol.flatten())
print(testy[20:30])

In [ ]:
testy[:10]

In [ ]:
predict(model, test)

## Tensorflow implementation

In [ ]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import tempfile
import os

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
nHidden = config['train']['inputNeurons']

tf_model = keras.Sequential([
    keras.layers.Dense(units=16, activation='relu'),
    keras.layers.Dense(units=nHidden[0], activation='relu'),
    keras.layers.Dense(units=nHidden[1], activation='relu'),
    keras.layers.Dense(units=nHidden[2], activation='relu'),
    keras.layers.Dense(units=nHidden[3], activation='sigmoid')  #activation='softmax'
])

In [ ]:
tf_model.compile(optimizer='adam', 
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = tf_model.fit(trainX, trainy, epochs=10)